In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # Output layer for binary classification (1 node)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Extract the last hidden state output
        out = out[:, -1, :]

        # Pass the last hidden state output to fully connected layer
        out = self.fc(out)
        # Apply sigmoid function to convert to a probability
        out = torch.sigmoid(out)
        return out

# Example usage:
input_size = 10   # Number of input features
hidden_size = 20  # Number of features in the hidden state
num_layers = 1    # Number of stacked RNN layers

# Create the RNN model instance
model = SimpleRNN(input_size, hidden_size, num_layers)

# Example input (batch_size, sequence_length, input_size)
x = torch.rand(5, 8, input_size)
y = torch.randint(0, 2, (5, 1)).type(torch.FloatTensor)
# Get the model output
output = model(x)
print(output)


tensor([[0.4183],
        [0.4078],
        [0.4335],
        [0.4133],
        [0.4174]], grad_fn=<SigmoidBackward0>)


# Task 1: Train the SimpleRNN using the Adam optimizer

In [11]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# wandb.init(project='dat550', reinit=True, name=f'run_adam')
# Training loop (for demonstration, let's say 100 epochs)
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    predicted_labels = model(x)

    # Compute and print loss
    loss = criterion(predicted_labels, y)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Print final loss
print(f'Final Loss: {loss.item()}')

Epoch 0, Loss: 0.0007844916544854641
Epoch 10, Loss: 9.726232383400202e-05
Epoch 20, Loss: 3.1612697057425976e-05
Epoch 30, Loss: 1.6729436538298614e-05
Epoch 40, Loss: 1.1660570635285694e-05
Epoch 50, Loss: 9.239933206117712e-06
Epoch 60, Loss: 7.81655398895964e-06
Epoch 70, Loss: 6.828654477430973e-06
Epoch 80, Loss: 6.053265678929165e-06
Epoch 90, Loss: 5.436839273897931e-06
Final Loss: 4.958887984685134e-06


In [3]:
pip install wandb

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import wandb
# wandb.login(key="your_api_key_here")
